In [2]:
import requests
import json
import torch
import torch.nn as nn
import os
from tqdm import tqdm
from transformers import BertModel, BertTokenizerFast, AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR
import matplotlib.pyplot as plt

2023-04-01 14:38:44.394486: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 14:38:44.527553: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-01 14:38:45.471180: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /software/spackages/linux-rocky8-x86_64/gcc-9.5.0/anaconda3-2022.05-zyrazrj6uvrtukupqzh

In [3]:
def get_data(path): 
    
    with open(path, 'rb') as f:
        raw_data = json.load(f)
    contexts = []
    questions = []
    answers = []
    num_q = 0
    num_pos = 0
    num_imp = 0
    for group in raw_data['data']:
        for paragraph in group['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                num_q  = num_q+1
                for answer in qa['answers']:
                    contexts.append(context.lower())
                    questions.append(question.lower())
                    answers.append(answer)
                    # if answer['is_impossible'] == False:
                    #     num_pos = num_pos + 1
                    # else:
                    #     num_imp = num_imp + 1
    return num_q, num_pos, num_imp, contexts, questions, answers

In [4]:
num_of_questions = 0
num_of_possible = 0
num_of_impossible = 0
num_q, num_pos, num_imp, train_contexts, train_questions, train_answers = get_data('spoken_train-v1.1.json')
num_of_questions  = num_q
num_of_possible = num_pos
num_of_impossible  = num_imp
# print(num_q,num_pos,num_imp)
num_q, num_pos, num_imp, valid_contexts, valid_questions, valid_answers = get_data('spoken_test-v1.1.json')
# print(num_q,num_pos,num_imp)

In [5]:
# modifies each answer dictionary in place by converting the text to lowercase and adds
# key 'answer_end' which represents the ending position of the answer in the context.
def add_answer_at_end(answers, contexts):
    for answer, context in zip(answers, contexts):
        answer['text'] = answer['text'].lower()
        answer['answer_end'] = answer['answer_start'] + len(answer['text'])

add_answer_at_end(train_answers, train_contexts)
add_answer_at_end(valid_answers, valid_contexts)

#  maximum length of the input sequences after tokenization
MAX_LENGTH = 250
MODEL_PATH = "bert-base-uncased"
# The tokenizer is used to encode the training and validation questions and contexts into input sequences that can be fed into a BERT model.
tokenizerFast = BertTokenizerFast.from_pretrained(MODEL_PATH)

# Encoded dictionaries returned  contain the encoded inputs for the BERT model, including the input IDs, 
doc_stride = 128 
# attention masks, and token type IDs. 
train_encodings_fast = tokenizerFast(train_questions, train_contexts,  max_length = MAX_LENGTH, truncation=True, padding=True, stride = doc_stride)
valid_encodings_fast = tokenizerFast(valid_questions,valid_contexts,  max_length = MAX_LENGTH, truncation=True, padding=True, stride = doc_stride)

# The 'input_ids' list contains the token IDs for the encoded input sequence, including special tokens like [CLS] and [SEP]. The input sequence has been padded with zeros to the maximum length specified.

# The 'attention_mask' list is a binary mask indicating which tokens in the input sequence are padding tokens (indicated by a value of 0) and which are actual tokens (indicated by a value of 1).

# The 'token_type_ids' list is used to distinguish between the question and context portions of the input sequence. For example, in a question-answering task, the question would be represented with 0 values in this list, and the context would be represented with 1 values.



In [6]:
def return_Answer_startandend_train(idx):
    return_start = 0
    return_end = 0
    answer_encoding_fast = tokenizerFast(train_answers[idx]['text'],  max_length = MAX_LENGTH, truncation=True, padding=True)
    for a in range( len(train_encodings_fast['input_ids'][idx]) -  len(answer_encoding_fast['input_ids']) ): 
        match = True
        for i in range(1,len(answer_encoding_fast['input_ids']) - 1):
            if (answer_encoding_fast['input_ids'][i] != train_encodings_fast['input_ids'][idx][a + i]):
                match = False
                break
            if match:
                return_start = a+1
                return_end = a+i+1
                break
    return(return_start, return_end)


In [7]:
start_positions = []
end_positions = []
counter = 0
for t in range(len(train_encodings_fast['input_ids'])):
    s,e = return_Answer_startandend_train(t)
    start_positions.append(s)
    end_positions.append(e)
    if s==0:
        counter = counter + 1
train_encodings_fast.update({'start_positions': start_positions, 'end_positions': end_positions})
print(counter)
print(len(train_encodings_fast))


478
5


In [8]:
print(train_encodings_fast[1])

Encoding(num_tokens=250, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [9]:
# Accessing the first record in train_encodings_fast
encoding = train_encodings_fast[0]

# Accessing the attributes of the encoding
ids = encoding.ids
tokens = encoding.tokens
offsets = encoding.offsets
attention_mask = encoding.attention_mask
special_tokens_mask = encoding.special_tokens_mask

# Printing the results
print("IDs: ", ids)
print("Tokens: ", tokens)
print("Offsets: ", offsets)
print("Attention Mask: ", attention_mask)
print("Special Tokens Mask: ", special_tokens_mask)


IDs:  [101, 2054, 2003, 1999, 2392, 1997, 1996, 10289, 8214, 2364, 2311, 1029, 102, 6549, 2135, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 2082, 8514, 2003, 1996, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1999, 5307, 2009, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 10439, 14995, 6924, 2007, 1996, 5722, 1998, 1996, 2919, 2033, 5004, 3415, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 10047, 2984, 1999, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, 23052, 2012, 10223, 26371, 2605, 2073, 1996, 6261, 2984, 22353, 2135, 2596, 2000, 2358, 16595, 9648, 4674, 2145, 5255, 7763, 5595, 2809, 1012, 2012, 1996, 2203, 1997, 1996, 2364, 3298, 1998, 1999, 1037, 3622, 2240, 2008, 8539, 2083, 2093, 11342, 1999, 1996, 2751, 8514, 2003, 2004, 3722, 2715, 2962, 6231, 1997, 2984, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [10]:
# The same for text dataset
def return_answer_startend_valid(idx):
    return_start = 0
    return_end = 0
    answer_encoding_fast = tokenizerFast(valid_answers[idx]['text'],  max_length = MAX_LENGTH, truncation=True, padding=True)
    for a in range( len(valid_encodings_fast['input_ids'][idx])  -  len(answer_encoding_fast['input_ids'])   ): 
        match = True
        for i in range(1,len(answer_encoding_fast['input_ids']) - 1):
            if (answer_encoding_fast['input_ids'][i] != valid_encodings_fast['input_ids'][idx][a + i]):
                match = False
                break
            if match:
                return_start = a+1
                return_end = a+i+1
                break
    return(return_start, return_end)

In [11]:
start_positions = []
end_positions = []
counter = 0
for h in range(len(valid_encodings_fast['input_ids']) ):
   
    s, e = return_answer_startend_valid(h)
    start_positions.append(s)
    end_positions.append(e)
    if s==0:
        counter = counter + 1

valid_encodings_fast.update({'start_positions': start_positions, 'end_positions': end_positions})
print(counter)


236


In [12]:
class InputDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, i):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][i]),
            'token_type_ids': torch.tensor(self.encodings['token_type_ids'][i]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][i]),
            'start_positions': torch.tensor(self.encodings['start_positions'][i]),
            'end_positions': torch.tensor(self.encodings['end_positions'][i])
        }
    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = InputDataset(train_encodings_fast)
valid_dataset = InputDataset(valid_encodings_fast)

train_data_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=1)

bert_model = BertModel.from_pretrained(MODEL_PATH)  #MODEL_PATH = "bert-base-uncased"


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
class QAModel(nn.Module):
    def __init__(self):
        super(QAModel, self).__init__()
        self.bert = bert_model
        self.drop_out = nn.Dropout(0.1)
        self.l1 = nn.Linear(768 * 2, 768 * 2)
        self.l2 = nn.Linear(768 * 2, 2)
        self.linear_relu_stack = nn.Sequential(
            self.drop_out,
            self.l1,
            nn.LeakyReLU(),
            self.l2 
        )
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        model_output = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, output_hidden_states=True)
        hidden_states = model_output[2]
        out = torch.cat((hidden_states[-1], hidden_states[-3]), dim=-1)  
        logits = self.linear_relu_stack(out)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

model = QAModel()


In [14]:
def loss_function(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)/2
    return total_loss

def focal_loss_function(start_logits, end_logits, start_positions, end_positions, gamma):
    smax = nn.Softmax(dim=1)
    probs_start = smax(start_logits)
    inv_probs_start = 1 - probs_start
    probs_end = smax(end_logits)
    inv_probs_end = 1 - probs_end
    lsmax = nn.LogSoftmax(dim=1)
    log_probs_start = lsmax(start_logits)
    log_probs_end = lsmax(end_logits)
    
    nll = nn.NLLLoss()
    
    fl_start = nll(torch.pow(inv_probs_start, gamma)* log_probs_start, start_positions)
    fl_end = nll(torch.pow(inv_probs_end, gamma)*log_probs_end, end_positions)
    return ((fl_start + fl_end)/2)

optim = AdamW(model.parameters(), lr=2e-5, weight_decay=2e-2)
scheduler = ExponentialLR(optim, gamma=0.9)
total_acc = []
total_loss = []

/home/sdulam/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
def train_epoch(model, dataloader, epoch):
    model = model.train()
    losses = []
    acc = []
    counter = 0
    batch_tracker = 0
    for batch in tqdm(dataloader, desc = 'Running Epoch '):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        out_start, out_end = model(input_ids=input_ids, 
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
        
        loss = focal_loss_function(out_start, out_end, start_positions, end_positions,1) 
        losses.append(loss.item())
        loss.backward()
        optim.step()
        
        start_predictor = torch.argmax(out_start, dim=1)
        end_pred = torch.argmax(out_end, dim=1)
            
        acc.append(((start_predictor == start_positions).sum()/len(start_predictor)).item())
        acc.append(((end_pred == end_positions).sum()/len(end_pred)).item())
       
        batch_tracker = batch_tracker + 1
        if batch_tracker==250 and epoch==1:
            total_acc.append(sum(acc)/len(acc))
            loss_avg = sum(losses)/len(losses)
            total_loss.append(loss_avg)
            batch_tracker = 0
    scheduler.step()
    ret_acc = sum(acc)/len(acc)
    ret_loss = sum(losses)/len(losses)
    return(ret_acc, ret_loss)


In [16]:
def evaluate_model(model, dataloader):
    model = model.eval()
    losses = []
    acc = []
    counter = 0
    answer_list=[]
    with torch.no_grad():
        for batch in tqdm(dataloader, desc = 'Running the Evaluation'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            start_true = batch['start_positions'].to(device)
            end_true = batch['end_positions'].to(device)
            
            out_start, out_end = model(input_ids=input_ids, attention_mask=attention_mask,token_type_ids=token_type_ids)
            
            start_predictor = torch.argmax(out_start)
            end_pred = torch.argmax(out_end)
            answer = tokenizerFast.convert_tokens_to_string(tokenizerFast.convert_ids_to_tokens(input_ids[0][start_predictor:end_pred]))
            tanswer = tokenizerFast.convert_tokens_to_string(tokenizerFast.convert_ids_to_tokens(input_ids[0][start_true[0]:end_true[0]]))
            answer_list.append([answer,tanswer])
        
    return answer_list

In [18]:
from evaluate import load
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
model.to(device)

wer = load("wer")
EPOCHS = 1
model.to(device)
wer_list=[]
for epoch in range(EPOCHS):
    train_acc, train_loss = train_epoch(model, train_data_loader, epoch+1)
    print(f"Train Accuracy: {train_acc}      Train Loss: {train_loss}")
    answer_list = evaluate_model(model, valid_data_loader)
    pred_answers=[]
    true_answers=[]
    for i in range(len(answer_list)):
        if(len(answer_list[i][0])==0):
            answer_list[i][0]="$"
        if(len(answer_list[i][1])==0):
            answer_list[i][1]="$"
        pred_answers.append(answer_list[i][0])
        true_answers.append(answer_list[i][1])
    wer_score = wer.compute(predictions=pred_answers, references=true_answers)
    wer_list.append(wer_score)
print(wer_list)

tokens = tokenizerFast.tokenize("This is a sentence.")
print(tokens)
output = tokenizerFast.convert_tokens_to_string(tokens)
print(output)

Running Epoch : 100%|██████████| 2320/2320 [06:53<00:00,  5.61it/s]


Train Accuracy: 0.613750769711774      Train Loss: 1.137192325281172


Running the Evaluation: 100%|██████████| 15875/15875 [01:49<00:00, 144.50it/s]


[3.763716535433071]
['this', 'is', 'a', 'sentence', '.']
this is a sentence.


In [56]:
# # Save model weights
# torch.save(model.state_dict(), 'model_weights.pt')

# # Load model weights
# model.load_state_dict(torch.load('model_weights.pt'))

# # Make 

In [21]:
f= open("spoken_test-v1.1.json")
test_data = json.load(f)
f.close()

In [22]:
test_dataset = test_data['data']

In [23]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re
    
    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [24]:
def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
  
      # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
  
    common_tokens = set(pred_tokens) & set(truth_tokens)
  
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
  
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
  
    return 2 * (prec * rec) / (prec + rec)
     

In [43]:
import numpy as np
f1_scores=[]

model.eval()
# models = [model] + [model.from_pretrained('bert-base-uncased').to(device) for _ in range(3-1)]

for i, example in enumerate(test_dataset):
    for qa_pair in example['paragraphs']:
        context = qa_pair['context']
        for qas in qa_pair['qas']:
            inputs = tokenizerFast.encode_plus(qas['question'],context, max_length=MAX_LENGTH, return_tensors='pt').to(device)
            start_logits = torch.zeros((1, len(context)))
            end_logits = torch.zeros((1, len(context)))
            #for model in models:f
            out_start, out_end = model(**inputs)
            start_predictor = torch.argmax(out_start)
            end_pred = torch.argmax(out_end)
            answer = tokenizerFast.convert_tokens_to_string(tokenizerFast.convert_ids_to_tokens(inputs['input_ids'][0][start_predictor:end_pred]))
            
            
#             outputs = model(**inputs)
#             answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
#             answer_end = torch.argmax(outputs[1]) + 1 
#             answer = tokenizerFast.convert_tokens_to_string(tokenizerFast.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
#             
            int(normalize_text(answer) == normalize_text(qas['answers'][0]['text']))
            f1_scores.append(compute_f1(answer, qas['answers'][0]['text']))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [45]:
overall_f1_score = np.mean(f1_scores)
print(overall_f1_score)

0.35046021759984486


In [40]:
# # Ensemble Predictions
# import numpy as np
# f1_scores=[]

# model.eval()
# # models = [model] + [model.from_pretrained('bert-base-uncased').to(device) for _ in range(3-1)]

# for i, example in enumerate(test_dataset):
#     for qa_pair in example['paragraphs']:
#         context = qa_pair['context']
#         for qas in qa_pair['qas']:
#             inputs = tokenizerFast.encode_plus(qas['question'],context, return_tensors='pt').to(device)
#             #inputs = {
#                 #'input_ids': torch.tensor(tokenized['input_ids']).unsqueeze(0).to(device),
#                 #'attention_mask': torch.tensor(tokenized['attention_mask']).unsqueeze(0).to(device)             
#             #}
#             start_logits = torch.zeros((1, len(context)))
#             end_logits = torch.zeros((1, len(context)))
#             #for model in models:f
#             out_start, out_end = model(**inputs)
#             start_predictor = torch.argmax(out_start)
#             end_pred = torch.argmax(out_end)
#             answer = tokenizerFast.convert_tokens_to_string(tokenizerFast.convert_ids_to_tokens(inputs['input_ids'][0][start_predictor:end_pred]))
            
            
# #             outputs = model(**inputs)
# #             answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
# #             answer_end = torch.argmax(outputs[1]) + 1 
# #             answer = tokenizerFast.convert_tokens_to_string(tokenizerFast.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
# #             
#             int(normalize_text(answer) == normalize_text(qas['answers'][0]['text']))
#             f1_scores.append(compute_f1(answer, qas['answers'][0]['text']))
         
# # input_ids = batch['input_ids'].to(device)
# #             attention_mask = batch['attention_mask'].to(device)
# #             token_type_ids = batch['token_type_ids'].to(device)
# #             start_true = batch['start_positions'].to(device)
# #             end_true = batch['end_positions'].to(device)
            
# #             out_start, out_end = model(input_ids=input_ids, attention_mask=attention_mask,token_type_ids=token_type_ids)
            
# #             start_predictor = torch.argmax(out_start)
# #             end_pred = torch.argmax(out_end)
# #             answer = tokenizerFast.convert_tokens_to_string(tokenizerFast.convert_ids_to_tokens(input_ids[0][start_predictor:end_pred]))
# #             tanswer = tokenizerFast.convert_tokens_to_string(tokenizerFast.convert_ids_to_tokens(input_ids[0][start_true[0]:end_true[0]]))
# #             answer_list.append([answer,tanswer])
# # overall_f1_score = np.mean(f1_scores)
# # print(overall_f1_score)
# #         '''
# #         start_logits += outputs.start_logits
# #         end_logits += outputs.end_logits
# #         start_logits /= ensemble_size
# #         end_logits /= ensemble_size
# #         start_index = torch.argmax(start_logits, dim=1).item()
# #         end_index = torch.argmax(end_logits, dim=1).item()
# #         answer = context[start_index:end_index]
# #         print(f'Example {i+1}: {answer}')
# #         '''

RuntimeError: The size of tensor a (579) must match the size of tensor b (512) at non-singleton dimension 1

In [92]:
valid_data_loader2 = DataLoader(valid_dataset, batch_size=16)

In [94]:
model.eval()

acc = []
start_true = 0
end_true = 0
for batch in tqdm(valid_data_loader2):
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)

        # outputs = model(input_ids, attention_mask=attention_mask)
        out_start, out_end = model(input_ids=input_ids, attention_mask=attention_mask,token_type_ids=token_type_ids)
            
        start_pred = torch.argmax(out_start, dim=1)
        end_pred = torch.argmax(out_end, dim=1)

        acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        acc.append(((end_pred == end_true).sum()/len(end_pred)).item())

acc = sum(acc)/len(acc)

print("\n\nT/P\tanswer_start\tanswer_end\n")
for i in range(len(start_true)):
    print(f"true\t{start_true[i]}\t{end_true[i]}\n"
        f"pred\t{start_pred[i]}\t{end_pred[i]}\n")
     

100%|██████████| 993/993 [00:54<00:00, 18.20it/s]



T/P	answer_start	answer_end

true	77	78
pred	77	78

true	77	78
pred	77	78

true	77	78
pred	77	78

